# GETTING GEOSPATIAL DATA THROUGH REST APIS IN PYTHON NOTEBOOK

Description: Learn how to access geospatial data through the City of Regina's Open GIS endpoints

[Blog Post](https://peparhugo.github.io/data-science/geospatial-data-through-apis.html)


## Import Libraries

In [3]:
import requests
import pandas as pd
import geopandas as gpd
import shapely

## Test Tree Web App API

In [74]:
tree_app_url='https://opengis.regina.ca/arcgis/rest/services/CGISViewer/TreeWebApp/MapServer/0/query?outFields=*'
resp=requests.get(url=tree_app_url,
                 params=dict(
                     f='json',
                     returnGeometry='true',
                     spatialRel='esriSpatialRelIntersects',
                     where='(1=1) AND (1=1)',
                     orderByFields='GLOBALID ASC',
                     outSR=4326,
                     resultOffset=0,
                     resultRecordCount=1
                 ))
resp.json()

{'displayFieldName': 'STREETNAMEFULL',
 'fieldAliases': {'OBJECTID': 'OBJECTID',
  'YEARINSTALLED': 'YEARINSTALLED',
  'SPECIES': 'SPECIES',
  'NOTES': 'NOTES',
  'DBH': 'DBH',
  'TREEVALUE': 'TreeValue',
  'OWNER': 'OWNER',
  'SIGNIFICANCE_TREE': 'SIGNIFICANCE_TREE',
  'GLOBALID': 'GLOBALID',
  'SOIL_AMENDMENT': 'SOIL_AMENDMENT'},
 'geometryType': 'esriGeometryPoint',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'OBJECTID',
   'type': 'esriFieldTypeOID',
   'alias': 'OBJECTID'},
  {'name': 'YEARINSTALLED',
   'type': 'esriFieldTypeInteger',
   'alias': 'YEARINSTALLED'},
  {'name': 'SPECIES',
   'type': 'esriFieldTypeString',
   'alias': 'SPECIES',
   'length': 50},
  {'name': 'NOTES',
   'type': 'esriFieldTypeString',
   'alias': 'NOTES',
   'length': 1000},
  {'name': 'DBH', 'type': 'esriFieldTypeSmallInteger', 'alias': 'DBH'},
  {'name': 'TREEVALUE', 'type': 'esriFieldTypeDouble', 'alias': 'TreeValue'},
  {'name': 'OWNER',
   'type': 'esriFieldTypeSt

## Retrieve all Tree Web App records

In [61]:
results = []
nbr_results = 1
while nbr_results>0:
    resp=requests.get(url=tree_app_url,
                     params=dict(
                         f='json',
                         returnGeometry='true',
                         spatialRel='esriSpatialRelIntersects',
                         where='(1=1) AND (1=1)',
                         orderByFields='GLOBALID ASC',
                         outSR=4326,
                         resultOffset=len(results),
                         resultRecordCount=10000
                     ))
    nbr_results=len(resp.json()['features'])
    results.extend(resp.json()['features'])
    print(len(results))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
143375
143375


In [62]:
len(results)

143375

## Convert JSON records into dataframe

In [75]:
data = pd.DataFrame([{**doc['attributes'],"geometry":doc.get('geometry')} for doc in results])
data=data[data.geometry.isna()==False]
data.shape

(143373, 11)

## Convert dataframe into geodataframe

In [69]:
geodata = gpd.GeoDataFrame(data.drop(columns=['geometry']),
                           geometry=data.geometry.map(lambda x: shapely.geometry.Point(float(x['x']),
                                                                                      float(x['y']))))
geodata.head()

,OBJECTID,YEARINSTALLED,SPECIES,NOTES,DBH,TREEVALUE,OWNER,SIGNIFICANCE_TREE,GLOBALID,SOIL_AMENDMENT,geometry
0,112557,NaN,American Elm,None,38.0,13626.15,City,N,{00003627-F4BF-4684-8016-E3FE69BEDDBD},N,POINT (-104.59693 50.40507)
1,551767,NaN,American Elm,None,28.0,7398.13,City,None,{00007D45-7F68-4517-850F-A19588A465E3},N,POINT (-104.62509 50.41370)
2,569980,NaN,Poplar,None,48.0,14420.35,City,None,{00008661-7277-4C4D-941F-A6008E109651},N,POINT (-104.67830 50.45861)
3,296854,NaN,Green Ash,Centre median,29.0,7693.07,City,None,{0000D103-3553-40F2-9617-7713D4B02203},N,POINT (-104.60520 50.44078)
4,299752,NaN,American Elm,Side Boulevard,63.0,37453.05,City,None,{00011879-CE3A-4170-9EB2-C546019DF5A2},N,POINT (-104.60085 50.44707)


## Retrieve all Neighbourhood records

In [77]:
resp=requests.get(url="https://opengis.regina.ca/arcgis/rest/services/CGISViewer/Neighbourhood_Profile/MapServer/0/query?outFields=*",
                     params=dict(
                         f='json',
                         returnGeometry='true',
                         spatialRel='esriSpatialRelIntersects',
                         where='(1=1) AND (1=1)',
                         outSR=4326,
                         resultOffset=0,
                         resultRecordCount=10000
                     ))
resp.json()['features'][0:1]

[{'attributes': {'OBJECTID': 1,
   'CA': 'WHITMORE PARK',
   'NEIGH_AREA': ' ',
   'Shape_Length': 7504.10496471471,
   'Shape_Area': 2936200.4533456834,
   'PDF_Link': 'https://regina.ca/about-regina/neighbourhood-profiles/.galleries/pdfs/whitmore-park.pdf'},
  'geometry': {'rings': [[[-104.60905929785373, 50.415484024136795],
     [-104.60904849233098, 50.411883493529444],
     [-104.60030508176132, 50.411887900916504],
     [-104.60030517330979, 50.411046775773706],
     [-104.59784252726828, 50.41044205125727],
     [-104.59571523346489, 50.40954850106288],
     [-104.59564430865197, 50.40961735762469],
     [-104.5952625400897, 50.40989199803777],
     [-104.59440614601492, 50.41034860298679],
     [-104.5942343353836, 50.4102063677372],
     [-104.59408080049744, 50.410079260935355],
     [-104.59390699951967, 50.40994766740371],
     [-104.59375398512901, 50.40985189429068],
     [-104.59356404811005, 50.40974281118223],
     [-104.59338334476092, 50.40965782652863],
     [-104.

## Create geodataframe

In [79]:
data_nbh = pd.DataFrame([{**doc['attributes'],"geometry":doc.get('geometry')} for doc in resp.json()['features']])
geo_nbh = gpd.GeoDataFrame(data_nbh.drop(columns=['geometry']),
                           geometry=data_nbh.geometry.map(lambda doc: shapely.geometry.Polygon(doc['rings'][0])))
geo_nbh.head()

,OBJECTID,CA,NEIGH_AREA,Shape_Length,Shape_Area,PDF_Link,geometry
0,1,WHITMORE PARK,,7504.104965,2.936200e+06,https://regina.ca/about-regina/neighbourhood-p...,"POLYGON ((-104.60906 50.41548, -104.60905 50.4..."
1,2,CENTRE SQUARE,,3085.068648,5.928274e+05,https://regina.ca/about-regina/neighbourhood-p...,"POLYGON ((-104.61370 50.44077, -104.61806 50.4..."
2,3,UPLANDS,,8385.793704,3.581797e+06,https://regina.ca/about-regina/neighbourhood-p...,"POLYGON ((-104.59498 50.50618, -104.59499 50.4..."
3,4,TWIN LAKES,,7206.786977,3.260129e+06,https://regina.ca/about-regina/neighbourhood-p...,"POLYGON ((-104.67508 50.51329, -104.67508 50.5..."
4,5,PRAIRIE VIEW,,8671.983244,3.971544e+06,https://regina.ca/about-regina/neighbourhood-p...,"POLYGON ((-104.69842 50.48434, -104.68789 50.4..."
